In [107]:
import json
# import modules
import os
from openai import OpenAI
from dotenv import load_dotenv
from pprint import pprint

# load the .env file
load_dotenv()

True

In [108]:
# Initializing the OpenAi client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEYS"), project=os.environ.get("OPENAI_PROJECT_ID"))

In [109]:
# define the initial messages
# Lets motivate openai to make use of its tools and add a message that tells it to use its tools to help the user solve math problems
messages = [
    {"role": "system", "content": "You are a math assistant, use your tools to help the user solve math problems"},
    {"role": "user", "content": "What is 7 to the power of 34?"},
    # {"role": "user", "content": "What is 7 to the power of 34? And also what is the square of 7?"},
    # {"role": "user", "content": "What is the meaning of life?"},
]

In [110]:
# define a function that can help us calculate the exponentiation
# We will call this function when openai asks us to.
def to_the_power(base, exponent):
    return str(base ** exponent)

In [111]:
# we will define a tool list that describes our function
# pick good descriptions so openai has a good understanding of what function we have and what it can do.
tools = [
    {
        "type": "function",
        "function": {
            "name": "to_the_power",
            "description": "Calculate exponentiation result by raising the base to the power of the exponent",
            "parameters": {
                "type": "object",
                "properties": {
                    "base": {
                        "type": "integer",
                        "description": "The base, that will be raised to the power",
                    },
                    "exponent": {
                        "type": "integer",
                        "description": "The exponent or power that the base will be raised to",
                    },
                },
                "required": ["base", "exponent"],
            },
        }
    }
]

In [112]:
# Send these messages to the OpenAI API for chat completion
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    tools=tools,
    temperature=0
)

# inspect the response message
response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jwgWI6w8Kqi3Rq89lyUByBY2', function=Function(arguments='{"base": 7, "exponent": 34}', name='to_the_power'), type='function'), ChatCompletionMessageToolCall(id='call_ud4vrelnVv5M2An3BjW3D6hZ', function=Function(arguments='{"base": 7, "exponent": 2}', name='to_the_power'), type='function')])

In [113]:
# let's append the response to the messages list
messages.append(response.choices[0].message)
messages

[{'role': 'system',
  'content': 'You are a math assistant, use your tools to help the user solve math problems'},
 {'role': 'user',
  'content': 'What is 7 to the power of 34? And also what is the square of 7?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jwgWI6w8Kqi3Rq89lyUByBY2', function=Function(arguments='{"base": 7, "exponent": 34}', name='to_the_power'), type='function'), ChatCompletionMessageToolCall(id='call_ud4vrelnVv5M2An3BjW3D6hZ', function=Function(arguments='{"base": 7, "exponent": 2}', name='to_the_power'), type='function')])]

In [114]:
if response.choices[0].message.tool_calls:  # if there are tool calls in the response
    for tool_call in response.choices[0].message.tool_calls: # iterate over the tool calls
        if tool_call.function.name == "to_the_power": # if the tool call is for the to_the_power function
            kwargs = tool_call.function.arguments # extract the arguments
            kwargs = json.loads(kwargs) # convert the arguments to a dictionary
            result = to_the_power(**kwargs) # call the function with the arguments **unpacks the dictionary
            call_id = tool_call.id # extract the call id
            # append the result as a message to the message list
            messages.append({
                "role":"tool", 
                "tool_call_id":call_id, 
                "name": tool_call.function.name, 
                "content": result
            })
    pprint(messages)

[{'content': 'You are a math assistant, use your tools to help the user solve '
             'math problems',
  'role': 'system'},
 {'content': 'What is 7 to the power of 34? And also what is the square of 7?',
  'role': 'user'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jwgWI6w8Kqi3Rq89lyUByBY2', function=Function(arguments='{"base": 7, "exponent": 34}', name='to_the_power'), type='function'), ChatCompletionMessageToolCall(id='call_ud4vrelnVv5M2An3BjW3D6hZ', function=Function(arguments='{"base": 7, "exponent": 2}', name='to_the_power'), type='function')]),
 {'content': '54116956037952111668959660849',
  'name': 'to_the_power',
  'role': 'tool',
  'tool_call_id': 'call_jwgWI6w8Kqi3Rq89lyUByBY2'},
 {'content': '49',
  'name': 'to_the_power',
  'role': 'tool',
  'tool_call_id': 'call_ud4vrelnVv5M2An3BjW3D6hZ'}]


In [115]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0
)
print(response.choices[0].message.content)

('- 7 to the power of 34 is equal to 54,116,956,037,952,111,668,959,660,849.\n'
 '- The square of 7 is 49.')


# Exercise
Next match of your favorite football team.
## Step 1: Get your apilayer API key
Apilayer is a service that groups free APIs. We will use the Google search API from apilayer. But feel free to pick 
your own if you are interested in something else.
Go to https://apilayer.com/ create an account, log in. Then hover over your user icon on the top right, go to Account,
and you will see your API key there, copy it and paste it in the .env file.
## Step 2: Get your openai API key
If you have an account at openai get your API key and also add it to the .env file.
## Step 3: Test the tool
Run google.py to see if it returns the search results as expected.
## Step 4: DIY
* Go to main.py and complete the TODOs